In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
df = pd.read_csv('data/kc_house_data.csv')

Examined head of data, data types, and null values.

In [2]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

There are 6 columns that contain object data types. If kept, these will need to be converted in order to apply linear regression models.

In [4]:
df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

View, waterfront, and year renovated all contain null values. We will inspect each individually in order to determine if these will be dropped or recategorized.

In [5]:
df['waterfront'].value_counts()

NO     19075
YES      146
Name: waterfront, dtype: int64

In [6]:
#several of the rows contain null values, in order to preserve data
#we decided to recategorize null at "unknown"
df["waterfront"] = df["waterfront"].fillna("Unknown")

In [7]:
df['view'].value_counts()

NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [8]:
#after reading the description of view and seeing the large number of houses with no view 
#we decided that it is not a relevant column and will be dropped from the dataframe 
df = df.drop('view', axis=1)

In [9]:
df['yr_renovated'].value_counts()

0.0       17011
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64

In [10]:
# we assumed that 0 indicates that a house has not been renovated
# in order to keep the data we will change the nulls to 1 because 
# median value would change them all to 0 and the mean value was 83, which is not a real year
df["yr_renovated"] = df["yr_renovated"].fillna(1)

In [11]:
df['sqft_basement'].value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
1852.0        1
3500.0        1
374.0         1
1281.0        1
143.0         1
Name: sqft_basement, Length: 304, dtype: int64

We saw that there is a '?' as a value. This was not explained and did not add to our knowledge so we decided to add a column that calculated the square footage of basement by subtracting total square footage from square footabge above ground.

In [12]:
df['sqft_basment_calc'] = df['sqft_living'] - df['sqft_above']

In [13]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_basment_calc
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,Unknown,Average,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,0
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,Average,...,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,400
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,Average,...,770,0.0,1933,1.0,98028,47.7379,-122.233,2720,8062,0
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,Very Good,...,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,910
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,Average,...,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,0


To better address what features can increase the value of a house we decided to drop the following numerical values that are used for data entry purposes only:

id

date

For previously explained reasons we will be dropping the following columns as well:

sqft_basement
view

In [14]:
df.drop('id', axis=1, inplace=True)
df.drop('date', axis=1, inplace=True)
df.drop('sqft_basement', axis=1, inplace=True)

Used Ordinal Encoder to transform categorical string data in grade, condition, and waterfront into numerical data, for the purpose of running multivariate linear regressions.

In [15]:
grades = df[['grade']]
categories = [['3 Poor', '4 Low', '5 Fair', '6 Low Average', '7 Average', '8 Good', '9 Better', '10 Very Good', '11 Excellent', '12 Luxury', '13 Mansion']]
ords = OrdinalEncoder(categories=categories)
ords.fit(grades)
ords.transform(grades)

array([[4.],
       [4.],
       [3.],
       ...,
       [4.],
       [5.],
       [4.]])

In [16]:
grades_encoded = pd.DataFrame(
 
    ords.transform(grades),
    
    index=df.index
)

In [17]:
df.drop('grade', axis=1, inplace=True)

In [18]:
df = pd.concat([df, grades_encoded], axis=1)

In [19]:
df['grades'] = df[0]

In [20]:
df.drop(0, axis=1, inplace=True)

In [21]:
waterfront_e = df[['waterfront']]
categories1 = [['NO', 'Unknown', 'YES']]
ords1 = OrdinalEncoder(categories=categories1)
ords1.fit(waterfront_e)
ords1.transform(waterfront_e)

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]])

In [22]:
waterfront_encoded = pd.DataFrame(
   
    ords1.transform(waterfront_e),

    index=df.index
)

In [23]:
df.drop('waterfront', axis=1, inplace=True)
df = pd.concat([df, waterfront_encoded], axis=1)

In [24]:
df['waterfront'] = df[0]
df.drop(0, axis=1, inplace=True)

In [25]:
conditions_e = df[['condition']]
categories2 = [['Poor', 'Fair', 'Average', 'Good', 'Very Good']]
ords2 = OrdinalEncoder(categories=categories2)
ords2.fit(conditions_e)
ords2.transform(conditions_e)

array([[2.],
       [2.],
       [2.],
       ...,
       [2.],
       [2.],
       [2.]])

In [26]:
conditions_encoded = pd.DataFrame(
    
    ords2.transform(conditions_e),

    index=df.index
)


In [27]:
df.drop('condition', axis=1, inplace=True)
df = pd.concat([df, conditions_encoded], axis=1)

In [28]:
df['condition'] = df[0]
df.drop(0, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   price              21597 non-null  float64
 1   bedrooms           21597 non-null  int64  
 2   bathrooms          21597 non-null  float64
 3   sqft_living        21597 non-null  int64  
 4   sqft_lot           21597 non-null  int64  
 5   floors             21597 non-null  float64
 6   sqft_above         21597 non-null  int64  
 7   yr_built           21597 non-null  int64  
 8   yr_renovated       21597 non-null  float64
 9   zipcode            21597 non-null  int64  
 10  lat                21597 non-null  float64
 11  long               21597 non-null  float64
 12  sqft_living15      21597 non-null  int64  
 13  sqft_lot15         21597 non-null  int64  
 14  sqft_basment_calc  21597 non-null  int64  
 15  grades             21597 non-null  float64
 16  waterfront         215

Now all data is numeric and ready for EDA.

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   price              21597 non-null  float64
 1   bedrooms           21597 non-null  int64  
 2   bathrooms          21597 non-null  float64
 3   sqft_living        21597 non-null  int64  
 4   sqft_lot           21597 non-null  int64  
 5   floors             21597 non-null  float64
 6   sqft_above         21597 non-null  int64  
 7   yr_built           21597 non-null  int64  
 8   yr_renovated       21597 non-null  float64
 9   zipcode            21597 non-null  int64  
 10  lat                21597 non-null  float64
 11  long               21597 non-null  float64
 12  sqft_living15      21597 non-null  int64  
 13  sqft_lot15         21597 non-null  int64  
 14  sqft_basment_calc  21597 non-null  int64  
 15  grades             21597 non-null  float64
 16  waterfront         215

In [30]:
df.to_csv('cleaned_kc_house_data.csv', index=False)